In [1]:
##1. import libraries

In [2]:

import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
##2. read and  process data

In [4]:
df = pd.read_excel('default of credit card clients.xls', header=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

In [6]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [7]:
df.drop(columns=['ID'], inplace=True)
df.rename(columns={'PAY_0':'PAY_1'}, inplace=True)

In [8]:
#no missing values
df.isna().sum().sum()

0

In [9]:
df.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [10]:
#one hot encoding for sex, education, marriage and pay
df = pd.get_dummies(df, prefix=['SEX', 'EDU', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'], columns=['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'])

In [11]:
#train_test_split
df_X = df.drop(['default payment next month'], axis=1)
df_y = df[['default payment next month']]

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=10)


In [12]:
col_to_scale = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

In [13]:
#standardize features 
from sklearn import preprocessing
std_scaler = preprocessing.StandardScaler()
X_train[col_to_scale] = std_scaler.fit_transform(X_train[col_to_scale])
X_test[col_to_scale] = std_scaler.transform(X_test[col_to_scale])

In [14]:
#Applying deep learning models

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, SimpleRNN

In [16]:
#reshape data
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
#SimpleRNN model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
rnn_model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
188/188 [==============================] - 0s 819us/step - loss: 0.4783 - accuracy: 0.7795
Epoch 2/100
188/188 [==============================] - 0s 730us/step - loss: 0.4374 - accuracy: 0.8159
Epoch 3/100
188/188 [==============================] - 0s 730us/step - loss: 0.4347 - accuracy: 0.8191
Epoch 4/100
188/188 [==============================] - 0s 715us/step - loss: 0.4339 - accuracy: 0.8198
Epoch 5/100
188/188 [==============================] - 0s 723us/step - loss: 0.4325 - accuracy: 0.8198
Epoch 6/100
188/188 [==============================] - 0s 727us/step - loss: 0.4311 - accuracy: 0.8208
Epoch 7/100
188/188 [==============================] - 0s 731us/step - loss: 0.4298 - accuracy: 0.8212
Epoch 8/100
188/188 [==============================] - 0s 718us/step - loss: 0.4296 - accuracy: 0.8202
Epoch 9/100
188/188 [==============================] - 0s 746us/step - loss: 0.4291 - accuracy: 0.8213
Epoch 10/100
188/188 [==============================] - 0s 699us/step - l

In [21]:
rnn_model.evaluate(X_test, y_test)

188/188 [==============================] - 0s 422us/step - loss: 0.4342 - accuracy: 0.8200


[0.43423983454704285, 0.8199999928474426]

In [57]:
y_pred = (rnn_model.predict(X_test).ravel()>0.5)+0 
print('\nConfusion matrix for simpleRNN: \n', confusion_matrix(y_test, y_pred))


Confusion matrix for simpleRNN: 
 [[4420  263]
 [ 817  500]]


In [62]:
#2. LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(50, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

In [63]:
lstm_model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
188/188 [==============================] - 0s 3ms/step - loss: 0.4777 - accuracy: 0.7951
Epoch 2/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4404 - accuracy: 0.8158
Epoch 3/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4358 - accuracy: 0.8191
Epoch 4/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4333 - accuracy: 0.8196
Epoch 5/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8203
Epoch 6/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4318 - accuracy: 0.8220
Epoch 7/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4313 - accuracy: 0.8213
Epoch 8/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4308 - accuracy: 0.8221
Epoch 9/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4278 - accuracy: 0.8231
Epoch 10/100
188/188 [==============================] - 0s 2ms/step - loss: 0.4286 - accura

In [64]:
# evaluate model
lstm_model.evaluate(X_test, y_test)

188/188 [==============================] - 0s 689us/step - loss: 0.4423 - accuracy: 0.8218


[0.44233763217926025, 0.8218333125114441]

In [65]:
y_pred = (lstm_model.predict(X_test).ravel()>0.5)+0
print('\nConfusion matrix for LSTM: \n', confusion_matrix(y_test, y_pred))


Confusion matrix for LSTM: 
 [[4485  198]
 [ 871  446]]


In [51]:
#3. GRU
from keras.optimizers import SGD
gru_model = Sequential()
gru_model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), activation='tanh'))
gru_model.add(Dropout(0.2))
gru_model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
gru_model.add(Dropout(0.2))
gru_model.add(Dense(1, activation='sigmoid'))
gru_model.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='binary_crossentropy', metrics=['accuracy'])

In [52]:
gru_model.fit(X_train,y_train,epochs=50,batch_size=128, verbose=1)

Epoch 1/50
188/188 [==============================] - 0s 2ms/step - loss: 0.5224 - accuracy: 0.7745
Epoch 2/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4673 - accuracy: 0.7915
Epoch 3/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4493 - accuracy: 0.8064
Epoch 4/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4437 - accuracy: 0.8093
Epoch 5/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4420 - accuracy: 0.8129
Epoch 6/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4392 - accuracy: 0.8152
Epoch 7/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4392 - accuracy: 0.8161
Epoch 8/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4372 - accuracy: 0.8175
Epoch 9/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4359 - accuracy: 0.8173
Epoch 10/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4362 - accuracy: 0.8171

In [66]:
gru_model.evaluate(X_test, y_test)

188/188 [==============================] - 0s 670us/step - loss: 0.4219 - accuracy: 0.8258


[0.4218994677066803, 0.8258333206176758]

In [67]:
y_pred = (gru_model.predict(X_test).ravel()>0.5)+0
print('\nConfusion matrix for GRU: \n', confusion_matrix(y_test, y_pred))


Confusion matrix for GRU: 
 [[4466  217]
 [ 828  489]]
